In [1]:
import pandas as pd
import numpy
import os
import 

import ipyleaflet
import geojson
import re

In [2]:
aeroportos = pd.read_csv('https://www.anac.gov.br/dadosabertos/areas-de-atuacao/aerodromos/aerodromos-cadastro-de-aerodromos-publicos-csv',
                          delimiter=";",
                         encoding = "cp1252",
                         header = 2,
                         usecols = ['CÓDIGO OACI','NOME','LATITUDE','LONGITUDE','ALTITUDE',
                                    'DESIGNAÇÃO', 'COMPRIMENTO', 'LARGURA',
                                    'DESIGNAÇÃO.1', 'COMPRIMENTO.1', 'LARGURA.1',
                                    'DESIGNAÇÃO.2', 'COMPRIMENTO.2', 'LARGURA.2'])
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2
0,SBAA,CONCEIÇÃO DO ARAGUAIA,8° 20' 55'' S,49° 18' 11'' W,199 m,08/26,1800 m,30 m,-,-,-,-,-,-
1,SBAE,BAURU/AREALVA,22° 9' 28'' S,49° 4' 6'' W,594 m,17/35,2010 m,45 m,-,-,-,-,-,-
2,SBAQ,BARTOLOMEU DE GUSMÃO,21° 48' 16'' S,48° 8' 25'' W,711 m,17/35,1800 m,30 m,-,-,-,-,-,-
3,SBAR,SANTA MARIA,10° 59' 7'' S,37° 4' 24'' W,7 m,11/29,2200 m,45 m,-,-,-,-,-,-
4,SBAT,PILOTO OSVALDO MARQUES DIAS,9° 51' 59'' S,56° 6' 18'' W,289 m,04/22,2500 m,30 m,-,-,-,-,-,-
5,SBAU,ESTADUAL DARIO GUARITA,21° 8' 39'' S,50° 25' 35'' W,415 m,05/23,2120 m,35 m,-,-,-,-,-,-
6,SBAX,ROMEU ZEMA,19° 33' 38'' S,46° 57' 56'' W,999 m,15/33,1900 m,30 m,-,-,-,-,-,-
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,1° 23' 5'' S,48° 28' 44'' W,17 m,06/24,2800 m,45 m,02/20,1830 m,45 m,-,-,-
8,SBBG,COMANDANTE GUSTAVO KRAEMER,31° 23' 27'' S,54° 6' 35'' W,186 m,06/24,1500 m,30 m,14/32,1149 m,45 m,-,-,-
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,19° 51' 7'' S,43° 57' 2'' W,789 m,13/31,2364 m,45 m,-,-,-,-,-,-


Dados de metros (em string) para pés (em float)

In [3]:
def to_feet(x):
    if (x == '-'):
        return float('nan')
    else:
        return float(x.replace('m','').replace(',','.'))/0.3048

In [4]:
cols_in_m = ['ALTITUDE', 'COMPRIMENTO', 'COMPRIMENTO.1' , 'COMPRIMENTO.2']
for index in cols_in_m:
    aeroportos[index] = aeroportos[index].apply(to_feet)

Coordenadas para Decimal

In [5]:
def to_dec(x):
    parts = re.split('[°\']+',x)
    if 'N' in parts[3]:
        return (int(parts[0]) + int(parts[1])/60.0 + float(parts[2])/3600.0)
    else:
        return -(int(parts[0]) + int(parts[1])/60.0 + float(parts[2])/3600.0)

In [6]:
cols_in_deg = ['LATITUDE', 'LONGITUDE']
for index in cols_in_deg:
    aeroportos[index] = aeroportos[index].apply(to_dec)
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2
0,SBAA,CONCEIÇÃO DO ARAGUAIA,-8.348611,-49.303056,652.887139,08/26,5905.511811,30 m,-,NaN,-,-,NaN,-
1,SBAE,BAURU/AREALVA,-22.157778,-49.068333,1948.818898,17/35,6594.488189,45 m,-,NaN,-,-,NaN,-
2,SBAQ,BARTOLOMEU DE GUSMÃO,-21.804444,-48.140278,2332.677165,17/35,5905.511811,30 m,-,NaN,-,-,NaN,-
3,SBAR,SANTA MARIA,-10.985278,-37.073333,22.965879,11/29,7217.847769,45 m,-,NaN,-,-,NaN,-
4,SBAT,PILOTO OSVALDO MARQUES DIAS,-9.866389,-56.105000,948.162730,04/22,8202.099738,30 m,-,NaN,-,-,NaN,-
5,SBAU,ESTADUAL DARIO GUARITA,-21.144167,-50.426389,1361.548556,05/23,6955.380577,35 m,-,NaN,-,-,NaN,-
6,SBAX,ROMEU ZEMA,-19.560556,-46.965556,3277.559055,15/33,6233.595801,30 m,-,NaN,-,-,NaN,-
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,-1.384722,-48.478889,55.774278,06/24,9186.351706,45 m,02/20,6003.937008,45 m,-,NaN,-
8,SBBG,COMANDANTE GUSTAVO KRAEMER,-31.390833,-54.109722,610.236220,06/24,4921.259843,30 m,14/32,3769.685039,45 m,-,NaN,-
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,-19.851944,-43.950556,2588.582677,13/31,7755.905512,45 m,-,NaN,-,-,NaN,-


In [7]:
points = [tuple(x) for x in aeroportos[['LONGITUDE','LATITUDE']].values]
dados = geojson.MultiPoint(points)
m = ipyleaflet.Map(center = (-15.8267,-47.9218), zoom = 5)
m.add_layer(ipyleaflet.GeoJSON(data=dados))
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [8]:
aeroportos['COMPRIMENTO_MAX'] = aeroportos[['COMPRIMENTO', 'COMPRIMENTO.1', 'COMPRIMENTO.2']].max(axis=1)
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2,COMPRIMENTO_MAX
0,SBAA,CONCEIÇÃO DO ARAGUAIA,-8.348611,-49.303056,652.887139,08/26,5905.511811,30 m,-,NaN,-,-,NaN,-,5905.511811
1,SBAE,BAURU/AREALVA,-22.157778,-49.068333,1948.818898,17/35,6594.488189,45 m,-,NaN,-,-,NaN,-,6594.488189
2,SBAQ,BARTOLOMEU DE GUSMÃO,-21.804444,-48.140278,2332.677165,17/35,5905.511811,30 m,-,NaN,-,-,NaN,-,5905.511811
3,SBAR,SANTA MARIA,-10.985278,-37.073333,22.965879,11/29,7217.847769,45 m,-,NaN,-,-,NaN,-,7217.847769
4,SBAT,PILOTO OSVALDO MARQUES DIAS,-9.866389,-56.105000,948.162730,04/22,8202.099738,30 m,-,NaN,-,-,NaN,-,8202.099738
5,SBAU,ESTADUAL DARIO GUARITA,-21.144167,-50.426389,1361.548556,05/23,6955.380577,35 m,-,NaN,-,-,NaN,-,6955.380577
6,SBAX,ROMEU ZEMA,-19.560556,-46.965556,3277.559055,15/33,6233.595801,30 m,-,NaN,-,-,NaN,-,6233.595801
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,-1.384722,-48.478889,55.774278,06/24,9186.351706,45 m,02/20,6003.937008,45 m,-,NaN,-,9186.351706
8,SBBG,COMANDANTE GUSTAVO KRAEMER,-31.390833,-54.109722,610.236220,06/24,4921.259843,30 m,14/32,3769.685039,45 m,-,NaN,-,4921.259843
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,-19.851944,-43.950556,2588.582677,13/31,7755.905512,45 m,-,NaN,-,-,NaN,-,7755.905512


In [9]:
voos = pd.read_csv('dados/diario.csv',
                    delimiter = ';',
                    usecols = ['Cód. Empresa','Nr. Voo','Equip.',
                               'Seg','Ter','Qua','Qui','Sex','Sáb','Dom',
                               'Cód Origem','Cód Destino'])
voos

,Cód. Empresa,Nr. Voo,Equip.,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Cód Origem,Cód Destino
0,AAF,0035,A332,0,0,0,0,0,6,0,LFPO,SBKP
1,AAF,0035,A332,0,2,0,4,5,0,7,LFPO,SBKP
2,AAF,0036,A332,0,0,0,0,0,6,0,SBKP,LFPO
3,AAF,0036,A332,0,2,0,4,5,0,7,SBKP,LFPO
4,AAL,0213,B752,1,2,3,4,5,6,7,KMIA,SBBR
5,AAL,0214,B752,1,2,3,4,5,6,7,SBBR,KMIA
6,AAL,0215,B789,0,2,0,4,0,6,7,KLAX,SBGR
7,AAL,0216,B789,0,2,0,4,0,6,7,SBGR,KLAX
8,AAL,0904,B772,1,2,3,4,5,6,7,SBGL,KMIA
9,AAL,0905,B772,1,2,3,4,5,6,7,KMIA,SBGL


In [10]:
# Salva Todos os Tipos de Aeronave
# Manualmente os Tipos São Inseridos
equip = voos['Equip.'].unique()
equip = numpy.sort(equip, axis = -1, kind='mergesort')
equip.tofile('dados/equip.txt', sep='\n', format='%s')

In [11]:
equip = pd.read_csv('dados/equip.csv',
                        delimiter = ',')

In [13]:
result = pd.merge(voos,equip,how='inner',on=['Equip.'])
result2 = pd.merge(result,aeroportos[['CÓDIGO OACI','COMPRIMENTO_MAX']],
                   how='inner',
                   left_on=['Cód Origem'],right_on=['CÓDIGO OACI'])
result2

,Cód. Empresa,Nr. Voo,Equip.,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Cód Origem,Cód Destino,Type,CÓDIGO OACI,COMPRIMENTO_MAX,LATITUDE,LONGITUDE
0,AAF,0036,A332,0,0,0,0,0,6,0,SBKP,LFPO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
1,AAF,0036,A332,0,2,0,4,5,0,7,SBKP,LFPO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
2,AZU,8702,A332,0,2,0,4,5,0,0,SBKP,KFLL,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
3,AZU,8704,A332,1,2,3,4,5,6,7,SBKP,KFLL,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
4,AZU,8750,A332,0,0,0,4,0,0,0,SBKP,LPPT,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
5,AZU,8752,A332,0,0,0,4,0,0,7,SBKP,LPPT,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
6,TPA,4052,A332,0,2,3,0,0,0,0,SBKP,SKBO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
7,TPA,4052,A332,1,0,0,0,0,0,0,SBKP,SKBO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
8,TPA,4060,A332,0,0,0,0,0,0,7,SBKP,SKBO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444
9,TPA,4060,A332,0,0,0,4,0,0,0,SBKP,SKBO,Wide Body,SBKP,10629.921260,-23.006944,-47.134444


In [28]:
apt_com_voos = result2.groupby('Cód Origem', as_index = False)['LATITUDE','LONGITUDE'].first()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [14]:
test = result2[(result2['Type'] == 'Single Aisle') & 
        (result2['COMPRIMENTO_MAX'] < 6000.0) & 
        (result2['Cód Origem'] != 'SBRJ')]
test

,Cód. Empresa,Nr. Voo,Equip.,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Cód Origem,Cód Destino,Type,CÓDIGO OACI,COMPRIMENTO_MAX,LATITUDE,LONGITUDE
3069,TAM,3006,A319,1,2,3,4,5,0,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3070,TAM,3294,A319,0,0,0,0,0,6,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3071,TAM,3474,A319,0,0,0,0,0,0,7,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3072,TAM,3737,A319,1,2,3,4,5,0,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3073,TAM,3764,A319,0,0,0,4,0,0,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3074,TAM,3929,A319,1,0,0,4,0,0,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3075,TAM,3185,A320,0,0,3,0,5,6,0,SBSR,SBGR,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3076,TAM,3185,A320,1,2,0,4,0,0,7,SBSR,SBGR,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3077,TAM,3225,A320,0,2,0,4,5,0,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722
3078,TAM,3427,A320,0,0,0,0,0,6,0,SBSR,SBSP,Single Aisle,SBSR,5380.577428,-20.816111,-49.404722


In [15]:
dias = ['Seg', 'Ter','Qua','Qui','Sex','Sáb','Dom']
apts = test['Cód Origem'].unique()
apt_crit = []
for i in apts:
    flag = False
    for d in dias:
        flag = flag | (test[d][test['Cód Origem'] == i] == 0).all()
    if(~flag):
        apt_crit.append(i)
apt_crit

['SBSR', 'SBNF', 'SBJV', 'SBIL', 'SBIZ', 'SBJU', 'SBMQ', 'SBCX']

In [45]:
# Initialize Map
m = ipyleaflet.Map(center = (-15.8267,-47.9218), zoom = 5)

# Divide Airport Critic from Not Critic
apt_crit_geo = apt_com_voos[apt_com_voos['Cód Origem'].isin(apt_crit)]
apt_ncrit_geo = apt_com_voos[~apt_com_voos['Cód Origem'].isin(apt_crit)]


# Plot Airport Critic from Not Critic
points = [tuple(x) for x in apt_crit_geo[['LONGITUDE','LATITUDE']].values]
dados = geojson.MultiPoint(points)
lay_crit = ipyleaflet.GeoJSON(data=dados, style = {'color': 'red'})
m.add_layer(lay_crit)

# Plot Airport Critic from Not Critic
points1 = [tuple(x) for x in apt_ncrit_geo[['LONGITUDE','LATITUDE']].values]
dados1 = geojson.MultiPoint(points1)
lay_ncrit = ipyleaflet.GeoJSON(data=dados1, style = {'visible': 'false'})
m.add_layer(lay_ncrit)

m.add_control(ipyleaflet.LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …